In [3]:
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import scipy.io as sio
import pickle

features = sio.loadmat('selectedFeatures.mat')
x_train = features['trainFeatures']
y_train = [item[0] for item in features['trainLabels']]

x_test = features['testFeatures']
y_test = features['testLabels']

## SVM

In [ ]:
svm = svm.SVC(verbose=True)
svm.fit(x_train, y_train)
with open('svm.pickle', 'wb') as handle:
    pickle.dump(svm, handle, protocol=pickle.HIGHEST_PROTOCOL)

[LibSVM]

In [5]:
with open('svm.pickle', 'wb') as handle:
    pickle.dump(svm, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('svm.pickle', 'rb') as handle:
    svm = pickle.load(handle)
    
svm.score(x_test, y_test)

0.30714285714285716

## LDA

In [ ]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
with open('lda.pickle', 'wb') as handle:
    pickle.dump(lda, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('svm.pickle', 'rb') as handle:
    svm = pickle.load(handle)
    
lda.score(x_test, y_test)

## NN

In [ ]:
nn = 